# MongoDB Atlas Search: Basic Search

## Setup

In [1]:
# import our libraries, instantiate our classes

import sys
import os
from pymongo import MongoClient
from dotenv import load_dotenv
import pprint

# Load environment variables from a .env file, overriding any existing variables in the environment.
# This is useful for not cluttering the code with sensitive information like API keys and database URIs.
load_dotenv(override=True)

MONGO_URI = os.environ["MONGO_URI"]

client = MongoClient(MONGO_URI)['sample_mflix']

pp = pprint.PrettyPrinter(indent=1)

### Simple search

#### Atlas Search Index

```json

{
  "mappings": {
    "dynamic": true
  }
}

```

In [2]:
basic_search = [
    {
        '$search' :{
              'text': {
                  'query': "fight club",
                  'path': "title"
              }
        }
    },
    {
        '$project' : {
            '_id' : 0,
            'title' : 1,
            'plot' : 1
        } 
    },
    {'$limit' : 5}
]

result = list(client['movies'].aggregate(basic_search))
pp.pprint(result)

[{'plot': 'An insomniac office worker, looking for a way to change his life, '
          'crosses paths with a devil-may-care soap maker, forming an '
          'underground fight club that evolves into something much, much '
          'more...',
  'title': 'Fight Club'},
 {'plot': 'When Zatoichi witnesses the murder of a young mother he promises to '
          'deliver the baby to the father. Along the way he gains the help of '
          'a reluctant criminal.',
  'title': 'Fight, Zatoichi, Fight'},
 {'plot': 'When Zatoichi witnesses the murder of a young mother he promises to '
          'deliver the baby to the father. Along the way he gains the help of '
          'a reluctant criminal.',
  'title': 'Fight, Zatoichi, Fight'},
 {'plot': 'This documentary follows the 2002 mayoral campaign in Newark, New '
          'Jersey in which the Cory Booker attempted to unseat longtime mayor '
          'Sharpe James.',
  'title': 'Street Fight'},
 {'plot': 'A video of an assault on a teenage

#### Now with Fuzzy

In [3]:
pipeline = [
    {
        '$search': {
            'text': {
                'query': "fihgt club", #Fight club in intentionally misspelled
                'path': "title",
                'fuzzy': {
                    'maxEdits': 2
                }
            }
        }
    },
    {
        '$project' : {
            '_id' : 0,
            'title' : 1,
            'plot' : 1
        } 
    },
    {'$limit' : 5}
]

result = list(client['movies'].aggregate(pipeline))
pp.pprint(result)

[{'plot': 'An insomniac office worker, looking for a way to change his life, '
          'crosses paths with a devil-may-care soap maker, forming an '
          'underground fight club that evolves into something much, much '
          'more...',
  'title': 'Fight Club'},
 {'plot': 'Reunited by the death of a college friend, three divorced women '
          'seek revenge on the husbands who left them for younger women.',
  'title': 'The First Wives Club'},
 {'plot': 'When a teenager learns that his next door neighbour is a vampire, '
          'no one will believe him.',
  'title': 'Fright Night'},
 {'plot': 'The story of a fastidiously organized businessman, whose '
          'perspective on life is forever changed through an unexpected '
          'encounter with a tiny fledgling bird.',
  'title': 'First Flight'},
 {'plot': 'A teenager suspects that his new neighbor is a vampire.',
  'title': 'Fright Night'}]


#### Now with Score

In [4]:
pipeline = [
    {
        '$search': {
            'text': {
                'query': "fihgt club",
                'path': "title",
                'fuzzy': {
                    'maxEdits': 2
                }
            }
        }
    },
    {
        '$project' : {
            '_id' : 0,
            'title' : 1,
            'plot' : 1,
            'score' : {"$meta" : "searchScore" }
        } 
    },
    {'$limit' : 5}
]

result = list(client['movies'].aggregate(pipeline))
pp.pprint(result)

[{'plot': 'An insomniac office worker, looking for a way to change his life, '
          'crosses paths with a devil-may-care soap maker, forming an '
          'underground fight club that evolves into something much, much '
          'more...',
  'score': 4.897842884063721,
  'title': 'Fight Club'},
 {'plot': 'Reunited by the death of a college friend, three divorced women '
          'seek revenge on the husbands who left them for younger women.',
  'score': 3.3110339641571045,
  'title': 'The First Wives Club'},
 {'plot': 'When a teenager learns that his next door neighbour is a vampire, '
          'no one will believe him.',
  'score': 3.0531017780303955,
  'title': 'Fright Night'},
 {'plot': 'The story of a fastidiously organized businessman, whose '
          'perspective on life is forever changed through an unexpected '
          'encounter with a tiny fledgling bird.',
  'score': 3.0531017780303955,
  'title': 'First Flight'},
 {'plot': 'A teenager suspects that his new neig

#### Now with autocomplete

#### Autocomplete index

```json

{
  "mappings": {
    "dynamic": true,
    "fields": {
      "title": [
        {
          "foldDiacritics": false,
          "maxGrams": 15,
          "minGrams": 2,
          "tokenization": "edgeGram",
          "type": "autocomplete"
        }
      ]
    }
  }
}

```

In [ ]:
pipeline = [
            {
              '$search': {
              'index' : "autocomplete",
                'autocomplete': {
                    'query': "figh",
                    'path': "title"
                }
              }
            },
            { '$limit' : 3 },
            { '$project' : { '_id' : 0, 'title' : 1}}
        ]

result = list(client['movies'].aggregate(pipeline))
pp.pprint(result)